##### Import Modules

In [1]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain.retrievers import BM25Retriever
from tools import CustomEmbeddings, process_file, extract_text_from_file

import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.embeddings.base import Embeddings
from PyPDF2 import PdfReader
import docx
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.prompts import ChatPromptTemplate

import glob

from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [2]:
files = glob.glob("*.pdf*")
print(files)

['38_Hannah Wolfe Resume.pdf', '38_Marketing Sales Intern_Job Description.pdf', '52_Garrett Cassells Resume.pdf', '52_Marketing Manager_Job Description.pdf']


##### Load Files

In [5]:
job_file = files[2]
resume_file = files[0]

In [6]:
job_file

'52_Garrett Cassells Resume.pdf'

In [5]:
# Simple passthrough function
# what is this function for?
def passthrough(input_data):
    return input_data

##### Extract job ad text

In [7]:
job_ad_text = extract_text_from_file(job_file)
print(job_ad_text)

Garrett Cassells
Remote Demand Generation, Sr. Digital Marketer, SEO, PPC, Analytics & Reporting
Atlanta, GA  30308
GarrettGCassells@gmail.com
+1 954 274 3326
Demand Generation, Performance Marketing, Digital Marketer, SEO, PPC & Analytics 
Dynamic Online Marketing Professional with extensive experience in creating compelling digital
marketing campaigns to build brands, attract customers and drive revenues. Knowledgeable in all
phases of online marketing, from ideation to execution. Skilled at creating strategies that incorporate
proven techniques and technologies to achieve a client organization’s objectives. History of delivering
results that help businesses expand their audience, generate awareness and increase sales.
Authorized to work in the US for any employer
Work Experience
Demand Generation – Performance Marketing Manager (Contract)
The Walt Disney Company (Disney Publishing – eCom)  - Remote
May 2021 to Present
• Google Ads, Microsoft Ads, Google Shopping, Paid Search, Amazon

##### Extract resume text


In [8]:
resume_text = extract_text_from_file(resume_file)
print(resume_text)

HANNAHWOLFE22006 Philip Drive | Leonardtown, MD 20650 | (301)904-2497 | hwolfe@muhlenberg.edu
EDUCATION
Leonar dtown Hig h School, Leonar dtown, MDGraduat edMay 2018High School Diploma
Muhlenber g College, Allent own, P AExpect ed Gr aduationMay 2022Bachelor of S cienc e, Major in Business A dministr ation 
●Concentr ation in Mar ketingBachelor of Arts, Media and C ommunications
●Current C umulati ve GP A: 3.415
SKILLS
Comput er Skills: 
●Experience using man y Adobe pr ograms such as Phot oshop,InDesign, Illustr ator 
●Ability t o dr aft fl yers, post ers, logos, T -shirt designs,cover art , etc. 
●Worked with HTML pr ograms such as R studio f or statisticalanal ysis 
●Proficient using Micr osoft Ex cel
WORKEXPERIENCE
Breton Ba y Country Club, Leonar dtown, MDJune 2017 – A ugust 2018 (seasonal)Lifeguar d 
●
  CPR Certified thr ough American R ed Cr oss 
●
  In char ge of the saf ety and health of patr ons atBreton Ba y pool, r equir ed team work with others aswellas ability t o tak e 

In [11]:
BASE_MODEL = '/home/james/models/mistralai/Mistral-7B-Instruct-v0.3'
# reference for CustomEmbeddings code?
embeddings = CustomEmbeddings(model_name=BASE_MODEL) 


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/james/miniconda3/lib/python3.12/site-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [12]:
# vector_store = FAISS.from_texts(job_ad_text, embedding=embeddings)
vector_store = FAISS.from_documents(chunks, embedding=embeddings)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


<class 'list'>


ValueError: too many values to unpack (expected 2)

In [9]:
file_path = job_file

loader = TextLoader("temp_text.txt")
data = loader.load()
chunk_size = 7500 #1000# 7500
chunk_overlap= 100 # 100
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = text_splitter.split_documents(data)

print (f"Your {len(data)} documents have been split into {len(chunks)} chunks")


Your 1 documents have been split into 1 chunks


In [ ]:
retriever = BM25Retriever(vector_db=job_ad_vectorstore.as_retriever())


In [11]:
chain = (
    {"context": retriever, "question": passthrough}
    | PROMPT
    | ChatOllama()
    | passthrough  # Simple output parsing
)
response = chain.invoke("Analyze the resume based on the job description")


Your 1 documents have been split into 4 chunks


In [24]:
# with open(file_path, "rb") as f:
#     reader = PdfReader(f)
#     text = ""
#     for page in reader.pages:
#         text += page.extract_text()
#     with open("temp_text.txt", "w") as text_file:
#         text_file.write(text)
#     loader = TextLoader("temp_text.txt")
#     data = loader.load()
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
#     chunks = text_splitter.split_documents(data)
#     if chunks:
#         embeddings_list = embeddings.embed_documents([chunk.page_content for chunk in chunks])
#         if embeddings_list:
#             vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [13]:
help(Chroma)

Help on class Chroma in module langchain_community.vectorstores.chroma:

class Chroma(langchain_core.vectorstores.VectorStore)
 |  Chroma(collection_name: 'str' = 'langchain', embedding_function: 'Optional[Embeddings]' = None, persist_directory: 'Optional[str]' = None, client_settings: 'Optional[chromadb.config.Settings]' = None, collection_metadata: 'Optional[Dict]' = None, client: 'Optional[chromadb.Client]' = None, relevance_score_fn: 'Optional[Callable[[float], float]]' = None) -> 'None'
 |
 |  `ChromaDB` vector store.
 |
 |  To use, you should have the ``chromadb`` python package installed.
 |
 |  Example:
 |      .. code-block:: python
 |
 |              from langchain_community.vectorstores import Chroma
 |              from langchain_community.embeddings.openai import OpenAIEmbeddings
 |
 |              embeddings = OpenAIEmbeddings()
 |              vectorstore = Chroma("langchain_store", embeddings)
 |
 |  Method resolution order:
 |      Chroma
 |      langchain_core.vectors

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name='/home/james/models/BAAI/bge-base-en-v1.5',
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

In [17]:
db = Chroma.from_documents(chunks, embeddings, persist_directory="db")

/home/james/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'us-api.i.posthog.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [1]:
# file_path = resume_file
file_path = job_file

with open(file_path, "rb") as f:
    reader = PdfReader(f)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    with open("temp_text.txt", "w") as text_file:
        text_file.write(text)

NameError: name 'job_file' is not defined

In [27]:
text

"Garrett Cassells\nRemote Demand Generation, Sr. Digital Marketer, SEO, PPC, Analytics & Reporting\nAtlanta, GA  30308\nGarrettGCassells@gmail.com\n+1 954 274 3326\nDemand Generation, Performance Marketing, Digital Marketer, SEO, PPC & Analytics\xa0\nDynamic Online Marketing Professional with extensive experience in creating compelling digital\nmarketing campaigns to build brands, attract customers and drive revenues. Knowledgeable in all\nphases of online marketing, from ideation to execution. Skilled at creating strategies that incorporate\nproven techniques and technologies to achieve a client organization’s objectives. History of delivering\nresults that help businesses expand their audience, generate awareness and increase sales.\nAuthorized to work in the US for any employer\nWork Experience\nDemand Generation – Performance Marketing Manager (Contract)\nThe Walt Disney Company (Disney Publishing – eCom)  - Remote\nMay 2021 to Present\n• Google Ads, Microsoft Ads, Google Shopping,

In [16]:
loader = TextLoader("temp_text.txt")
data = loader.load()
chunk_size = 100# 7500
chunk_overlap= 10 # 100
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = text_splitter.split_documents(data)

In [38]:
print (f"Your {len(data)} documents have been split into {len(chunks)} chunks")


Your 1 documents have been split into 74 chunks


In [39]:
type(chunks)

list

In [40]:
data

[Document(page_content="Garrett Cassells\nRemote Demand Generation, Sr. Digital Marketer, SEO, PPC, Analytics & Reporting\nAtlanta, GA  30308\nGarrettGCassells@gmail.com\n+1 954 274 3326\nDemand Generation, Performance Marketing, Digital Marketer, SEO, PPC & Analytics\xa0\nDynamic Online Marketing Professional with extensive experience in creating compelling digital\nmarketing campaigns to build brands, attract customers and drive revenues. Knowledgeable in all\nphases of online marketing, from ideation to execution. Skilled at creating strategies that incorporate\nproven techniques and technologies to achieve a client organization’s objectives. History of delivering\nresults that help businesses expand their audience, generate awareness and increase sales.\nAuthorized to work in the US for any employer\nWork Experience\nDemand Generation – Performance Marketing Manager (Contract)\nThe Walt Disney Company (Disney Publishing – eCom)  - Remote\nMay 2021 to Present\n• Google Ads, Microsof

In [41]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="job_file_docs",
    embedding_function=CustomEmbeddings(model_name=BASE_MODEL)
)

/home/james/miniconda3/lib/python3.12/site-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


/home/james/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'us-api.i.posthog.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [21]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore


In [42]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
# The storage layer for the parent documents
store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore, 
    docstore=store, 
    child_splitter=text_splitter,
)

In [32]:
type(data[0])

langchain_core.documents.base.Document

In [43]:
retriever.add_documents(data, ids=None)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [19]:
# embeddings_list = embeddings.embed_documents([chunk.page_content for chunk in chunks])


In [16]:
# vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [17]:
# db._collection.get(include=['embeddings'])
# embeddings.


In [20]:
# if chunks:
#     embeddings_list = embeddings.embed_documents([chunk.page_content for chunk in chunks])
#     if embeddings_list:
#         vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [13]:
# file_path = resume_file
file_path = job_file
with open(file_path, "rb") as f:
    reader = PdfReader(f)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    with open("temp_text.txt", "w") as text_file:
        text_file.write(text)
    loader = TextLoader("temp_text.txt")
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
    chunks = text_splitter.split_documents(data)
    if chunks:
        embeddings_list = embeddings.embed_documents([chunk.page_content for chunk in chunks])
        if embeddings_list:
            vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)


/home/james/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'us-api.i.posthog.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [14]:
job_vectorstore = vectorstore

In [15]:
file_path = resume_file
# file_path = job_file
with open(file_path, "rb") as f:
    reader = PdfReader(f)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    with open("temp_text.txt", "w") as text_file:
        text_file.write(text)
    loader = TextLoader("temp_text.txt")
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
    chunks = text_splitter.split_documents(data)
    if chunks:
        embeddings_list = embeddings.embed_documents([chunk.page_content for chunk in chunks])
        if embeddings_list:
            vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)


/home/james/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'us-api.i.posthog.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [16]:
resume_vectorstore = vectorstore

In [18]:
job_vectorstore.as_retriever()

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f50bfb385c0>)

In [19]:
# retriever = BM25Retriever(vector_db=job_vectorstore.as_retriever())


In [20]:
help(BM25Retriever)

Help on class BM25Retriever in module langchain_community.retrievers.bm25:

class BM25Retriever(langchain_core.retrievers.BaseRetriever)
 |  BM25Retriever(*, name: Optional[str] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, vectorizer: Any = None, docs: List[langchain_core.documents.base.Document], k: int = 4, preprocess_func: Callable[[str], List[str]] = <function default_preprocessing_func at 0x7f517dd46980>) -> None
 |
 |  `BM25` retriever without Elasticsearch.
 |
 |  Method resolution order:
 |      BM25Retriever
 |      langchain_core.retrievers.BaseRetriever
 |      langchain_core.runnables.base.RunnableSerializable
 |      langchain_core.load.serializable.Serializable
 |      pydantic.v1.main.BaseModel
 |      pydantic.v1.utils.Representation
 |      langchain_core.runnables.base.Runnable
 |      typing.Generic
 |      abc.ABC
 |      builtins.object
 |
 |  Class methods defined here:
 |
 |  from_documents(documents: 'Iterable[Document]', 

In [ ]:
# def process_file(file_path, embeddings):
#     if os.path.isfile(file_path):
        # with open(file_path, "rb") as f:
        #     reader = PdfReader(f)
        #     text = ""
        #     for page in reader.pages:
        #         text += page.extract_text()
        #     with open("temp_text.txt", "w") as text_file:
        #         text_file.write(text)
            # loader = TextLoader("temp_text.txt")
            # data = loader.load()
            # text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
            # chunks = text_splitter.split_documents(data)
            # if chunks:
            #     embeddings_list = embeddings.embed_documents([chunk.page_content for chunk in chunks])
            #     if embeddings_list:
            #         vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)
            #         return vectorstore
    # return None

In [21]:
# resume_processed = process_file(resume_file, embeddings)

In [21]:
from langchain.retrievers import BM25Retriever
BM25Retriever.

In [18]:
# Function to analyze inputs and return relevant information
def analyze_inputs(job_file_path, resume_file_path, job_ad_text, resume_text):
    if job_file_path and resume_file_path:
        # Use the local Mistral model
        # embeddings = CustomEmbeddings(model_name=BASE_MODEL)
        
        from langchain.embeddings import HuggingFaceEmbeddings

        embeddings = HuggingFaceEmbeddings(
            model_name='/home/james/models/BAAI/bge-base-en-v1.5',
            model_kwargs={"device": "cuda"},
            encode_kwargs={"normalize_embeddings": True},
        )

        job_ad_vectorstore = process_file(job_file_path, embeddings)
        resume_vectorstore = process_file(resume_file_path, embeddings)

        if job_ad_vectorstore and resume_vectorstore:
            # retriever = BM25Retriever(vector_db=job_ad_vectorstore.as_retriever())
            chain = (
                {"context": retriever, "question": passthrough}
                | PROMPT
                | ChatOllama()
                | passthrough  # Simple output parsing
            )
            response = chain.invoke("Analyze the resume based on the job description")
            return response

    return "Failed to pro

cess the files."

In [23]:
analyze_inputs(job_file, resume_file, job_ad_text, resume_text)

/home/james/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'us-api.i.posthog.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/james/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'us-api.i.posthog.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ValidationError: 1 validation error for BM25Retriever
docs
  field required (type=value_error.missing)

In [ ]:
from langchain_community.retrievers.bm25.


In [5]:


# result = analyze_inputs(uploaded_files, job_ad_text, resume_text)


In [8]:
# https://stackoverflow.com/questions/45480280/convert-scanned-pdf-to-text-python
# https://github.com/stanfordnlp/dspy/issues/835
# https://dspy-docs.vercel.app/docs/deep-dive/retrieval_models_clients/ChromadbRM
# https://stackoverflow.com/questions/77925185/valueerror-expected-embeddingfunction-call-to-have-the-following-signature
# https://github.com/gkamradt/langchain-tutorials/blob/main/data_generation/Advanced%20Retrieval%20With%20LangChain.ipynb


2024-06-28 16:28:37.762 
  command:

    streamlit run C:\Users\teres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-06-28 16:28:37.764 Session state does not function when running a script without `streamlit run`
